In [1]:
import Pkg
push!(LOAD_PATH, "../../");
Pkg.activate("mcmc_redistricting", shared=true)

using Revise
using RandomNumbers
using LiftedTreeWalk

  Activating project at `~/.julia/environments/mcmc_redistricting`


In [2]:
pctGraphPath = joinpath(".", "4x4.json")
precinct_name = "precinct"
county_name = "county"
population_col = "TOTPOP"
num_dists = 4
rng_seed = 454190
pop_dev = 0.0
cycle_walk_frac = 0.01
steps = 100000/cycle_walk_frac*2
edge_weights= "connections"

nodeData = Set([precinct_name, county_name, population_col])

base_graph = BaseGraph(
    pctGraphPath, 
    population_col, 
    inc_node_data=nodeData,
    edge_weights=edge_weights
);

graph = MultiLevelGraph(base_graph, [precinct_name]);

In [3]:
using Graphs, SimpleWeightedGraphs
""""""
function get_cut_edge_list(
    partition::LinkCutPartition;
    column::String="connections"
)
    graph = partition.graph
    num_dists = partition.num_dists
    raw_data = [zeros(Int64, num_dists) for _ = 1:num_dists]
    total = 0
    for e in edges(graph.simple_graph)
        n1, n2 = src(e), dst(e)
        d1, d2 = partition.node_to_dist[n1], partition.node_to_dist[n2]
        if d1 == d2
            continue
        end
        raw_data[d1][d2] += 1
        raw_data[d2][d1] += 1
    end
    data = Vector(undef, num_dists)
    for di = 1:num_dists
        ngb_counts = zeros(Int64, 5)
        ngb_data = raw_data[di]
        for ni = 1:num_dists
            ngb_counts[ngb_data[ni]+1] += 1
        end
        data[di] = Tuple(ngb_counts)
    end
    
    return Set(data)
end

get_cut_edge_list

In [4]:
# grab number of possible linking edge choices for each plan in list

plans = [(1, 1, 2, 2, 1, 1, 2, 2, 3, 3, 4, 4, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 4, 3, 3, 4, 4),
(1, 2, 3, 3, 1, 2, 3, 3, 1, 2, 4, 4, 1, 2, 4, 4),
(1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 4, 3, 1, 4, 4, 3),
(1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 2, 3, 3, 1, 2, 3, 3, 1, 2, 2, 4, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 2, 3, 3, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 3, 3, 2, 1, 3, 3, 4, 1, 4, 4, 4),
(1, 1, 2, 2, 3, 1, 1, 2, 3, 3, 4, 2, 3, 4, 4, 4),
(1, 2, 3, 3, 1, 2, 2, 3, 1, 4, 2, 3, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 3, 4, 2, 1, 3, 4, 4, 1, 3, 3, 4),
(1, 2, 2, 2, 1, 3, 2, 4, 1, 3, 3, 4, 1, 3, 4, 4),
(1, 2, 2, 2, 1, 3, 2, 4, 1, 3, 4, 4, 1, 3, 3, 4),
(1, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 4, 2, 3),
(1, 2, 2, 2, 1, 3, 3, 2, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 1, 2, 2, 3, 1, 2, 4, 3, 1, 2, 4, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 1, 2, 3, 4, 4, 2, 3, 3, 4, 4),
(1, 2, 2, 3, 1, 2, 4, 3, 1, 2, 4, 3, 1, 4, 4, 3),
(1, 2, 2, 2, 1, 2, 3, 3, 1, 1, 4, 3, 4, 4, 4, 3),
(1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4),
(1, 2, 2, 2, 1, 1, 2, 3, 1, 4, 3, 3, 4, 4, 4, 3)]

# I reordered so these aren't right
orbitsize = [1, 8, 4, 2, 8, 8, 4, 4, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 2, 2, 2]
cut_edges_per_plan = [8, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 
                      12, 12, 12, 12, 12, 12, 12, 12]
sp_trees = [256, 16, 16, 16, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

node_to_district = Dict{Tuple{Vararg{String}}, Int}()
partition_index = Dict()

for (pi, p) in enumerate(plans)
    for ii = 1:length(p)
        node_to_district[graph.id_to_partitions[1][ii]] = p[ii]
    end
    ml_partition = MultiLevelPartition(graph, node_to_district)
    rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
    partition = LinkCutPartition(ml_partition, rng);
    cut_edges = get_cut_edge_sum(partition)
    # log_linking_edges = exp(LiftedTreeWalk.get_log_linking_edges(partition))
    spanning_trees = Int(round(exp(get_log_spanning_forests(partition))))
    lst = get_cut_edge_list(partition)
    # if cut_edges == 11 && (1, 1, 1, 1, 0) in lst
    k = (cut_edges, get_cut_edge_list(partition))
    if haskey(partition_index, k)
        partition_index[k] = Tuple([collect(partition_index[k]); pi])
    else
        partition_index[k] = (pi,)
    end

    @show pi, cut_edges, get_cut_edge_list(partition)
    # end
end
partition_index

(pi, cut_edges, get_cut_edge_list(partition)) = (1, 8, Set(Any[(2, 0, 2, 0, 0)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (2, 10, Set(Any[(1, 1, 1, 0, 1), (1, 2, 1, 0, 0), (2, 0, 2, 0, 0), (2, 1, 0, 0, 1)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (3, 10, Set(Any[(1, 0, 2, 0, 1), (2, 0, 2, 0, 0), (3, 0, 0, 0, 1)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (4, 10, Set(Any[(1, 0, 3, 0, 0), (2, 0, 2, 0, 0)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (5, 11, Set(Any[(2, 0, 2, 0, 0), (1, 1, 1, 1, 0), (2, 0, 0, 2, 0)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (6, 11, Set(Any[(2, 1, 0, 1, 0), (1, 2, 0, 1, 0), (1, 0, 0, 3, 0)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (7, 11, Set(Any[(2, 0, 2, 0, 0), (1, 1, 1, 1, 0), (2, 0, 0, 2, 0)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (8, 11, Set(Any[(1, 2, 1, 0, 0), (1, 0, 1, 2, 0), (1, 2, 0, 1, 0)]))
(pi, cut_edges, get_cut_edge_list(partition)) = (9, 12, Set(Any[(1, 1, 0, 2, 0), (2, 0, 1, 1, 0), (

Dict{Any, Any} with 20 entries:
  (12, Set(Any[(1, 1, 0, 2, 0), (2, 0, 1, 1, 0), (2, 0, 0, 2, 0), (… => (9,)
  (12, Set(Any[(2, 1, 0, 1, 0), (1, 1, 1, 1, 0), (2, 0, 1, 1, 0), (… => (12,)
  (12, Set(Any[(2, 0, 0, 1, 1), (1, 1, 0, 1, 1), (2, 1, 0, 0, 1), (… => (14,)
  (12, Set(Any[(1, 1, 1, 0, 1), (2, 1, 0, 0, 1)]))                   => (17,)
  (11, Set(Any[(2, 0, 2, 0, 0), (1, 1, 1, 1, 0), (2, 0, 0, 2, 0)]))  => (5, 7)
  (10, Set(Any[(1, 0, 3, 0, 0), (2, 0, 2, 0, 0)]))                   => (4,)
  (12, Set(Any[(1, 0, 1, 1, 1), (2, 0, 2, 0, 0), (2, 1, 0, 0, 1), (… => (10,)
  (12, Set(Any[(2, 1, 0, 1, 0), (1, 0, 1, 2, 0), (1, 1, 1, 1, 0), (… => (13,)
  (12, Set(Any[(2, 0, 0, 1, 1), (2, 1, 0, 0, 1)]))                   => (16,)
  (11, Set(Any[(1, 2, 1, 0, 0), (1, 0, 1, 2, 0), (1, 2, 0, 1, 0)]))  => (8,)
  (8, Set(Any[(2, 0, 2, 0, 0)]))                                     => (1,)
  (12, Set(Any[(1, 2, 1, 0, 0), (2, 0, 1, 0, 1), (2, 1, 0, 0, 1), (… => (15,)
  (10, Set(Any[(1, 1, 1, 0, 1), (1,

In [114]:
# uniform measure on forests

constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]

measure = Measure()
# push_energy!(measure, get_log_spanning_forests, 0.0) 
            
part_key_counts = Dict{Tuple, Int64}()
edge_cut_counts = Dict{Int64, Int64}()
for ii = 1:steps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition)
    lst = get_cut_edge_list(partition)
    pi = partition_index[(cut_edges, lst)]
    part_key_counts[pi] = get(part_key_counts, pi, 0) + 1
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

println("run 1")
for (k, v) in part_key_counts
    tot_p = 0
    for ki in k
        tot_p += sp_trees[ki]*orbitsize[ki]/654
    end
    @show k, v, v/steps, tot_p
end

println()
ce_to_count = Dict(8=>256,10=>(128+64+32),11=>96,12=>78)
for (k, v) in edge_cut_counts
    tot_p = ce_to_count[k]/654
    @show k, v, v/steps, tot_p
end

run 1
(k, v, v / steps, tot_p) = ((4,), 917658, 0.0458829, 0.04892966360856269)
(k, v, v / steps, tot_p) = ((13,), 244954, 0.0122477, 0.012232415902140673)
(k, v, v / steps, tot_p) = ((21,), 55716, 0.0027858, 0.0030581039755351682)
(k, v, v / steps, tot_p) = ((2,), 4014555, 0.20072775, 0.19571865443425077)
(k, v, v / steps, tot_p) = ((15,), 240448, 0.0120224, 0.012232415902140673)
(k, v, v / steps, tot_p) = ((10,), 227981, 0.01139905, 0.012232415902140673)
(k, v, v / steps, tot_p) = ((18,), 114066, 0.0057033, 0.0061162079510703364)
(k, v, v / steps, tot_p) = ((16,), 122558, 0.0061279, 0.0061162079510703364)
(k, v, v / steps, tot_p) = ((12,), 235006, 0.0117503, 0.012232415902140673)
(k, v, v / steps, tot_p) = ((8,), 494580, 0.024729, 0.024464831804281346)
(k, v, v / steps, tot_p) = ((17,), 120375, 0.00601875, 0.0061162079510703364)
(k, v, v / steps, tot_p) = ((1,), 7924902, 0.3962451, 0.39143730886850153)
(k, v, v / steps, tot_p) = ((19,), 142820, 0.007141, 0.0061162079510703364)
(k, v,

In [5]:
# uniform measure on forests

constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]

measure = Measure()
push_energy!(measure, get_log_spanning_forests, 1.0) 
            
part_key_counts = Dict{Tuple, Int64}()
edge_cut_counts = Dict{Int64, Int64}()
for ii = 1:steps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition)
    lst = get_cut_edge_list(partition)
    pi = partition_index[(cut_edges, lst)]
    part_key_counts[pi] = get(part_key_counts, pi, 0) + 1
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

println("run 1")
for (k, v) in part_key_counts
    tot_p = 0
    for ki in k
        tot_p += orbitsize[ki]/117
    end
    @show k, v, v/steps, tot_p
end

println()
ce_to_count = Dict(8=>1,10=>14,11=>24,12=>78)
for (k, v) in edge_cut_counts
    tot_p = ce_to_count[k]/117
    @show k, v, v/steps, tot_p
end

run 1
(k, v, v / steps, tot_p) = ((4,), 1076463, 0.05382315, 0.017094017094017096)
(k, v, v / steps, tot_p) = ((15,), 672508, 0.0336254, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((21,), 171276, 0.0085638, 0.017094017094017096)
(k, v, v / steps, tot_p) = ((2,), 3637379, 0.18186895, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((10,), 689086, 0.0344543, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((18,), 367200, 0.01836, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((13,), 736336, 0.0368168, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((16,), 356885, 0.01784425, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((12,), 741387, 0.03706935, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((8,), 845500, 0.042275, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((17,), 372123, 0.01860615, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((1,), 1674958, 0.0837479, 0.008547008547008548)
(k, v, v / steps, tot_p) = ((19,), 378317, 0.01891585, 0.03418803418803419)
(k, v, v / ste

In [6]:
1≈1.0000000000001, 1≈2

(true, false)

In [ ]:
# uniform measure on forests

constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]

measure = Measure()
push_energy!(measure, get_log_spanning_forests, 1.0) 
            
part_key_counts = Dict{Tuple, Int64}()
edge_cut_counts = Dict{Int64, Int64}()
for ii = 1:steps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition)
    lst = get_cut_edge_list(partition)
    pi = partition_index[(cut_edges, lst)]
    part_key_counts[pi] = get(part_key_counts, pi, 0) + 1
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

println("run 1")
for (k, v) in part_key_counts
    tot_p = 0
    for ki in k
        tot_p += orbitsize[ki]/117
    end
    @show k, v, v/steps, tot_p
end

println()
ce_to_count = Dict(8=>1,10=>14,11=>24,12=>78)
for (k, v) in edge_cut_counts
    tot_p = ce_to_count[k]/117
    @show k, v, v/steps, tot_p
end

In [47]:
rng = PCG.PCGStateOneseq(UInt64, rng_seed + 40521591124)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

@show get_log_spanning_trees(partition)
cycle_walk = build_lifted_tree_cycle_walk(constraints)
p, update = cycle_walk(partition, rng)
@show p, update
de = LiftedTreeWalk.get_delta_energy(partition, measure, update)
@show de
oe = get_log_energy(partition, measure)
LiftedTreeWalk.update_partition!(partition, update)
ne = get_log_energy(partition, measure)
@show exp(oe-ne), exp(oe), exp(ne)

"getting log spanning trees" = "getting log spanning trees"
log_tree_data = LiftedTreeWalk.LogForestEnergyData(725609654221207811, [1.61895e-319, 1.5e-323, 1.0, 1.5e-323], [-1.0, -1.0, -1.0, -1.0])
partition.identifier = 725609654221207812
("no update loop", log_trees[ii]) = ("no update loop", 0.0)
("no update loop", log_trees[ii]) = ("no update loop", 0.0)
("no update loop", log_trees[ii]) = ("no update loop", 0.0)
("no update loop", log_trees[ii]) = ("no update loop", 0.0)
log_trees = [0.0, 0.0, 0.0, 0.0]
("returning", log_spanning_trees) = ("returning", [0.0, 0.0, 0.0, 0.0])
get_log_spanning_trees(partition) = [0.0, 0.0, 0.0, 0.0]
(p, update) = (6.0, LiftedTreeWalk.Update{Int64}((1, 3), [[9, 10], [2, 6]], [[9, 5], [10, 6]], Dict{Tuple{Int64, Int64}, Set{SimpleWeightedGraphs.SimpleWeightedEdge}}((1, 2) => Set([Edge 10 => 11 with weight 1.0]), (1, 3) => Set([Edge 10 => 6 with weight 1.0, Edge 9 => 5 with weight 1.0]), (1, 4) => Set([Edge 14 => 15 with weight 1.0]), (2, 3) => Set([Edge

(0.25, 1.0, 4.0)